In [1]:
# improts for our used libs
import utils
import torch
import pandas as pd
import numpy as np
import model_simple as ms

from torch import nn
from data_loader import get_loader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Create X for model and y for decision tree

In [2]:
timeframe = 30

In [ ]:
utils.create_X_y('PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', loss_ratio=3, timeframe=30)

In [ ]:
# now we want to create our csv files for X and for y (X = the input for our xgboost and y the output of our xgboost)
# remeber X will be also used for our Autoencoders
X = pd.read_csv('data/X_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv')
y = pd.read_csv('data/y_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv')

### Just for evaulation

In [ ]:
# check if same length in the table
print(X.shape[0] == y.shape[0])
# show me which values we got :-)
unique, counts = np.unique(y['2'], return_counts=True)
dict(zip(unique, counts))

## create model

In [ ]:
# timeframe is meant the shape of tensor regarding fields
# so can be diffrent from timeframe abofe
ts = 60*24
data = get_loader(data_path='data/X_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', timeframe=ts, batch_size=51280)
# set learning rate and input dim
lr = 1e-1
ds = iter(data)
input_dim = ds.next().shape[2]
# inizialise model
model = ms.SimpleAutoencoder(input_dim=input_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# model # to print my model

In [ ]:
# train model
epoch_num = 1
model.cuda()
model.train()
criterion.cuda()
for epoch in range(epoch_num):
    for i, x in enumerate(data):
        # forward
        x = x.float().cuda()
        y = model(x).float().cuda()
        loss = criterion(y, x)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ------- log output ---------
    print('epoch [{}/{}], loss:{:.6f}, MSE_loss:{:.6f}'
          .format(epoch + 1, epoch_num, loss.data, 0))

## evaluate and save model result

In [ ]:
dp = get_loader(data_path='data/X_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', timeframe=ts, batch_size=1)
# prefill results until first dataloader element
result = [0] * ts
criterion.cuda()
for i, x in enumerate(dp):
    # add value to pandas 
    pred = model(x.float().cuda())
    result.append(criterion(x.float().cuda(), pred).data.item())
    if(i % 10000 == 0):
        print('still running' + i)
pd.DataFrame(result).to_csv('data/ym__PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', header=None, index=None)

In [ ]:
pd.DataFrame(result).to_csv('data/ym___PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv', header=None, index=None)

## Decision tree

In [8]:
filename_X = "data/ym___PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv"
filename_y = 'data/y_PAH3DEEUR_1 Min_Bid_2008.10.21_2018.10.27.csv'
X = pd.read_csv(filename_X, header=None)
y = pd.read_csv(filename_y)

utils.xgEvaluation(X, y, y_field='4', i_end=50, debug_i=5, threshold_confusionm=0.5)

treshhold: 0.5 i: 0
[[389898      0]
 [  6920      0]]
treshhold: 0.5 i: 5
[[389898      0]
 [  6920      0]]
treshhold: 0.5 i: 10
[[389898      0]
 [  6920      0]]
treshhold: 0.5 i: 15
[[389898      0]
 [  6920      0]]
treshhold: 0.5 i: 20
[[389898      0]
 [  6916      4]]
treshhold: 0.5 i: 25
[[389896      2]
 [  6919      1]]
treshhold: 0.5 i: 30
[[389890      8]
 [  6909     11]]
treshhold: 0.5 i: 35
[[389897      1]
 [  6913      7]]
treshhold: 0.5 i: 40
[[389895      3]
 [  6909     11]]
treshhold: 0.5 i: 45
[[389892      6]
 [  6907     13]]
treshhold: 0.5 i: 49
[[389887     11]
 [  6907     13]]
